<a href="https://colab.research.google.com/github/rameshnatarajanus/Sentinel-5p-NO2-data-analysis/blob/main/Notebooks/Campd_EmissionsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install mapclassify
!pip install geodatasets
!pip install pystac-client
!pip install planetary_computer

In [3]:
import requests
import sys
from datetime import datetime
import pandas as pd
import geopandas as gpd
import mapclassify
import warnings
import numpy as np
import pprint
import json
import matplotlib.pyplot as plt
from ipywidgets import interact





# CAMPD Streaming services API (facilities and emissions data)


In [4]:
#@title Set your API key here

# Obtained from [here](https://www.epa.gov/power-sector/cam-api-portal#/api-key-signup)

API_KEY = 'BevnN1JoUAc8BMTXrnvo5i3jnDhkp8Pieec1HHYT'

Streaming API call. For streaming API's you can use the  'page' and  'perPage' input parameters if the results are to too large to obtain as a single request

Obligatory warning to use the bulk transfer API instead of the streaming API if the request size is too large (e.g. multi-years of data, emissions data from a single utility etc.)

In [5]:
#@title Function to obtain data set via streaming services API

print("Warning: The streaming services API may result in a bad request for large requests.")
print("Please consider using the bulk data api endpoint instead")

# collecting data from request as a data frame

def getStreamingResponse(streamingUrl, params):

  # ultimately log all api calls

  streamingResponse = requests.get(streamingUrl, params)

  # printing the response error message and exit if the response is not successful

  print("Success: Status code = "+str(streamingResponse.status_code))
  if (int(streamingResponse.status_code) > 399):
      sys.exit("Error message: "+streamingResponse.json()['error']['message'])

  # Note: these are only printed when paging is used.

  if 'X-Field-Mappings' in streamingResponse.headers:
    fieldMappings = streamingResponse.headers['X-Field-Mappings']
    print("Field Mappings: "+str(fieldMappings))

  # Note: only used for checking the validity of beginDate and endDate if used

  if 'beginDate' in params and 'endDate' in params:
    date_format = "%Y-%m-%d"
    beginDate = datetime.strptime(params['beginDate'], date_format)
    endDate = datetime.strptime(params['endDate'], date_format)
    if (endDate < beginDate):
      print("check the beginDate and endDate query fields for consistency")

  df = pd.DataFrame(streamingResponse.json())
  print(f"Returned {df.shape[0]} rows and {df.shape[1]} columns ")

  return df

Please consider using the bulk data api endpoint instead


# Facility/attributes dataset

## Data loading

The API query parameters for facility/attributes data are published [here](https://www.epa.gov/power-sector/cam-api-portal#/swagger/facilities-mgmt).


- **page** is page number
- **perPage** is number of results per page (ignored if page is not set)
- **facilityId** DOE Energy Information Administration Plant ID Code ("ORIS code")
- **stateCode** AL, AK, AS, AZ, AR, CA, CO, CT, DE, DC, FM, FL, GA, GU, HI, ID, IL, IN, IA, KS, KY, LA, ME, MH, MD, MA, MI, MN, MS, MO, MT, NE, NV, NH, NJ, NM, NY, NC, ND, MP, OH, OK, OR, PW, PA, PR, RI, SC, SD, TN, TX, UT, VT, VI, VA, WA, WV, WI, WY
- **unitType** Arch-fired boiler, Bubbling fluidized bed boiler, Cyclone boiler, Cell burner boiler, Combined cycle, Circulating fluidized bed boiler, Combustion turbine, Dry bottom wall-fired boiler, Dry bottom turbo-fired boiler, Dry bottom vertically-fired boiler, Internal combustion engine, Integrated gasification combined cycle, Cement Kiln, Other boiler, Other turbine, Pressurized fluidized bed boiler, Process Heater, Stoker, Tangentially-fired, Wet bottom wall-fired boiler, Wet bottom turbo-fired boiler, Wet bottom vertically-fired boiler
- **unitFuelType** Coal, Coal Refuse, Diesel Oil, Liquified Petroleum Gas, Natural Gas, Other Gas, Other Oil, Other Solid Fuel, Petroleum Coke, Pipeline Natural Gas, Process Gas, Process Sludge, Refuse, Residual Oil, Tire Derived Fuel, Waste Liquid, Wood
- **sourceCategory** Automotive Stampings, Bulk Industrial Chemical, Cement Manufacturing, Cogeneration, Electric Utility, Industrial Boiler, Industrial Turbine, Institutional, Iron & Steel, Municipal Waste Combustor, Pulp & Paper Mill, Petroleum Refinery, Portland Cement Plant, Small Power Producer, Theme Park
- **programCodeInfo** ARP, CAIRNOX, CAIROS, CAIRSO2, CSNOX, CSNOXOS, CSOSG1, CSOSG2, CSOSG2E, CSOSG3, CSSO2G1, CSSO2G2, NBP, NHNOX, NSPS4T, OTC, RGGI, SIPNOX, TXSO2



In [12]:

YEARS = '2023'
UNIT_FUEL_TYPES = 'Coal|Pipeline Natural Gas|Natural Gas'
SOURCE_CATEGORIES = 'Cogeneration|Electric Utility|Industrial Boiler|Industrial Turbine|Institutional'

facilityAttributesUrl = "https://api.epa.gov/easey/streaming-services/facilities/attributes"
facilityAttributes_params =  {
    'api_key': API_KEY,
    'year': YEARS,  # required field e.g. '2020|2021'
}
optional_params = {
    # 'stateCode': '',
    # 'facilityId': '',
    # 'unitType': '',
    'unitFuelType': UNIT_FUEL_TYPES,
    # 'controlTechnologies': '',
    'sourceCategory': SOURCE_CATEGORIES,
    # 'programCodeInfo': '',
    # 'page': 1,
    # 'perPage': 100
}

facilityAttributes_params.update(optional_params)

facilityAttributes_df = getStreamingResponse(facilityAttributesUrl, facilityAttributes_params)

print(facilityAttributes_df.dtypes)

Success: Status code = 200
Field Mappings: [{"label":"State","value":"stateCode"},{"label":"Facility Name","value":"facilityName"},{"label":"Facility ID","value":"facilityId"},{"label":"Unit ID","value":"unitId"},{"label":"Associated Stacks","value":"associatedStacks"},{"label":"Year","value":"year"},{"label":"Program Code","value":"programCodeInfo"},{"label":"Primary Rep Info","value":"primaryRepInfo"},{"label":"EPA Region","value":"epaRegion"},{"label":"NERC Region","value":"nercRegion"},{"label":"County","value":"county"},{"label":"County Code","value":"countyCode"},{"label":"FIPS Code","value":"fipsCode"},{"label":"Source Category","value":"sourceCategory"},{"label":"Latitude","value":"latitude"},{"label":"Longitude","value":"longitude"},{"label":"Owner/Operator","value":"ownerOperator"},{"label":"SO2 Phase","value":"so2Phase"},{"label":"NOx Phase","value":"noxPhase"},{"label":"Unit Type","value":"unitType"},{"label":"Primary Fuel Type","value":"primaryFuelInfo"},{"label":"Secondar

## Facility/attribute overall statistics

In [13]:


print(f"facilities dataset columns:\n {list(facilityAttributes_df.columns)}")
print(f"\nNo. of unique years:")
print(facilityAttributes_df.groupby('year').agg({'year': ['nunique',  'min', 'max']}))
print(f"\nNo. of unique facilities in dataset: {facilityAttributes_df['facilityId'].nunique()}")

print(f"\nNo. of facilities grouped by sourceCategory (facilities may have units in different categories):")
print(facilityAttributes_df[['facilityId', 'sourceCategory']].drop_duplicates().groupby(
                           'sourceCategory').agg(
                                { 'facilityId': 'count',}
                               )
      )

display(facilityAttributes_df)

facilities dataset columns:
 ['stateCode', 'facilityName', 'facilityId', 'unitId', 'associatedStacks', 'year', 'programCodeInfo', 'primaryRepInfo', 'epaRegion', 'nercRegion', 'county', 'countyCode', 'fipsCode', 'sourceCategory', 'latitude', 'longitude', 'ownerOperator', 'so2Phase', 'noxPhase', 'unitType', 'primaryFuelInfo', 'secondaryFuelInfo', 'so2ControlInfo', 'noxControlInfo', 'pmControlInfo', 'hgControlInfo', 'commercialOperationDate', 'operatingStatus', 'maxHourlyHIRate', 'associatedGeneratorsAndNameplateCapacity']

No. of unique years:
        year            
     nunique   min   max
year                    
2023       1  2023  2023

No. of unique facilities in dataset: 1233

No. of facilities grouped by sourceCategory (facilities may have units in different categories):
                    facilityId
sourceCategory                
Cogeneration               122
Electric Utility          1069
Industrial Boiler           40
Industrial Turbine           7
Institutional            

,stateCode,facilityName,facilityId,unitId,associatedStacks,year,programCodeInfo,primaryRepInfo,epaRegion,nercRegion,...,primaryFuelInfo,secondaryFuelInfo,so2ControlInfo,noxControlInfo,pmControlInfo,hgControlInfo,commercialOperationDate,operatingStatus,maxHourlyHIRate,associatedGeneratorsAndNameplateCapacity
0,AL,Barry,3,1,CS0AAN,2023,"ARP, CSNOX, CSOSG2, CSSO2G2",605449,4,SERC,...,Pipeline Natural Gas,None,None,Low NOx Burner Technology w/ Closed-coupled OF...,None,None,1954-02-12,Operating,2322.0,1 (153.1)
1,AL,Barry,3,2,CS0AAN,2023,"ARP, CSNOX, CSOSG2, CSSO2G2",605449,4,SERC,...,Pipeline Natural Gas,None,None,Low NOx Burner Technology w/ Closed-coupled OF...,None,None,1954-07-12,Operating,2359.0,2 (153.1)
2,AL,Barry,3,4,None,2023,"ARP, CSNOX, CSOSG2, CSSO2G2",605449,4,SERC,...,Pipeline Natural Gas,None,None,"Low NOx Burner Technology w/ Separated OFA,Sel...",None,None,1969-12-31,Operating,3768.0,4 (403.8)
3,AL,Barry,3,5,None,2023,"ARP, CSNOX, CSOSG2, CSSO2G2, MATS",605449,4,SERC,...,Coal,Pipeline Natural Gas,Wet Limestone,"Low NOx Burner Technology w/ Separated OFA,Sel...",Electrostatic Precipitator,"Catalyst (gold, palladium, or other) used to o...",1971-10-19,Operating,11057.0,5 (788.8)
4,AL,Barry,3,6A,None,2023,"ARP, CSNOX, CSOSG2, CSSO2G2",605449,4,SERC,...,Pipeline Natural Gas,None,None,"Dry Low NOx Burners,Selective Catalytic Reduction",None,None,2000-03-14,Operating,3000.0,"A1ST (191.8), A1CT (170.1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3732,OH,Pratt Paper (OH),880109,B001,None,2023,SIPNOX,"610155 (Ended Mar 01, 2023), 610587 (Started M...",5,None,...,Pipeline Natural Gas,None,None,"Combustion Modification/Fuel Reburning,Low NOx...",None,None,2019-09-25,Operating,332.3,None
3733,TN,Holston Army Ammunition Plant,880110,1,None,2023,SIPNOX,"610579 (Started Feb 21, 2023)(Ended Oct 12, 20...",4,None,...,Pipeline Natural Gas,Diesel Oil,None,"Low NOx Burner Technology (Dry Bottom only),Se...",Wet ESP,None,2021-12-18,Operating,327.0,None
3734,TN,Holston Army Ammunition Plant,880110,2,None,2023,SIPNOX,"610579 (Started Feb 21, 2023)(Ended Oct 12, 20...",4,None,...,Pipeline Natural Gas,Diesel Oil,None,"Low NOx Burner Technology (Dry Bottom only),Se...",Wet ESP,None,2021-12-21,Operating,327.0,None
3735,TN,Holston Army Ammunition Plant,880110,3,None,2023,SIPNOX,"610579 (Started Feb 21, 2023)(Ended Oct 12, 20...",4,None,...,Pipeline Natural Gas,Diesel Oil,None,"Low NOx Burner Technology (Dry Bottom only),Se...",Wet ESP,None,2021-09-20,Operating,327.0,None


## Data cleaning and transformation

In [14]:


# # Here is a list of returned columns
# ['stateCode', 'facilityName', 'facilityId', 'unitId', 'associatedStacks',
#        'year', 'programCodeInfo', 'primaryRepInfo', 'epaRegion', 'nercRegion',
#        'county', 'countyCode', 'fipsCode', 'sourceCategory', 'latitude',
#        'longitude', 'ownerOperator', 'so2Phase', 'noxPhase', 'unitType',
#        'primaryFuelInfo', 'secondaryFuelInfo', 'so2ControlInfo',
#        'noxControlInfo', 'pmControlInfo', 'hgControlInfo',
#        'commercialOperationDate', 'operatingStatus', 'maxHourlyHIRate',
#        'associatedGeneratorsAndNameplateCapacity']

# Step 1: Retain  required columns

req_columns = ['facilityName','facilityId', 'unitId', 'associatedStacks',
                'year', 'stateCode', 'countyCode','fipsCode', 'sourceCategory',
                'latitude', 'longitude',
                'primaryFuelInfo', 'secondaryFuelInfo', 'so2ControlInfo',
                'noxControlInfo', 'pmControlInfo', 'hgControlInfo','operatingStatus',
                'maxHourlyHIRate', 'associatedGeneratorsAndNameplateCapacity']


facilityAttributes_df = facilityAttributes_df[req_columns]

# Step 2: Retain only those units with operatingStatus='Operating'

facilityAttributes_df = facilityAttributes_df[
      facilityAttributes_df['operatingStatus'] == 'Operating'
    ].drop('operatingStatus', axis = 1)

# Step 3: Collaps all units from the same sourceCategory,
# concatenate values (eliminating duplicates for all units) for 'associatedStacks','primaryFuelInfo', 'secondaryFuelInfo', 'so2ControlInfo',
# 'noxControlInfo', 'pmControlInfo', 'hgControlInfo', 'associatedGeneratorsAndNameplateCapacity'
# TODO add 'associatedGeneratorsAndNameplateCapacity'

conv_col_to_list = lambda x: [str.strip (e) for e in str.split(x, ",")] if x is not None else None

facilityAttributes_df['associatedStacks'] = facilityAttributes_df['associatedStacks'].apply(
      conv_col_to_list
    )
facilityAttributes_df['associatedGeneratorsAndNameplateCapacity'] = facilityAttributes_df['associatedGeneratorsAndNameplateCapacity'].apply(
      conv_col_to_list
    )

temp_df = facilityAttributes_df.groupby(['facilityId', 'sourceCategory']).aggregate(
         nUnits = ("unitId", "count"),
         facilityName = ("facilityName", "first"),
         stateCode = ('stateCode', "first"),
         countyCode = ('countyCode', "first"),
         fipsCode = ('fipsCode', "first"),
         latitude = ("latitude",  "median"),
         longitude = ("longitude",  "median"),
         associatedStacks = ("associatedStacks", lambda x: ", ".join(
                                                     set([str(h) for g in filter(lambda e: e is not None,x) for h in g])))
    )
groupedFacilityAttributes_df = temp_df.reset_index().copy()
groupedFacilityAttributes_df

,facilityId,sourceCategory,nUnits,facilityName,stateCode,countyCode,fipsCode,latitude,longitude,associatedStacks
0,3,Electric Utility,8,Barry,AL,AL097,097,31.0069,-88.0103,CS0AAN
1,9,Electric Utility,1,Copper Station,TX,TX141,141,31.7569,-106.3750,
2,10,Electric Utility,11,Greene County,AL,AL063,063,32.6017,-87.7811,CS0EBN
3,26,Electric Utility,5,E C Gaston,AL,AL117,117,33.2442,-86.4567,"CS0CBN, MS5B, MS5A, CS0CAN"
4,47,Electric Utility,8,Colbert,AL,AL033,033,34.7439,-87.8486,
...,...,...,...,...,...,...,...,...,...,...
1220,880102,Electric Utility,2,"AES Puerto Rico, LP",PR,PR057,057,17.9477,-66.1540,
1221,880107,Industrial Boiler,3,ETMT Marcus Hook Terminal,PA,PA045,045,39.8076,-75.4239,
1222,880108,Industrial Boiler,2,Grain Processing Corporation,IN,IN027,027,38.6552,-87.1814,
1223,880109,Industrial Boiler,1,Pratt Paper (OH),OH,OH011,011,40.5426,-84.1916,


##  Plot facility locations

In [15]:
class  Facility:

    def __init__(self, attr_dict):
      if attr_dict is not None:
          for key, value in attr_dict.items():
              setattr(self, key.replace(' ', '_'), value)

    def __repr__(self):
      return " ".join([str(self.facilityId),self.facilityName,
                      self.stateCode, self.sourceCategory, f"({self.nUnits} units)"])

In [16]:
import folium
from folium.plugins import MarkerCluster
folium_ICONCOLORS =  ['red','blue', 'gray', 'darkred', 'lightred','orange',
    'beige', 'green','darkgreen', 'lightgreen', 'darkblue', 'lightblue',
    'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray', 'black'
]



# pick out the facility locations

temp_df = groupedFacilityAttributes_df[['facilityId', 'facilityName', 'stateCode', 'nUnits', 'sourceCategory',
                                           'latitude', 'longitude']].drop_duplicates()

# create the map
facility_map = folium.Map(location = [40, -102], zoom_start = 4, max_zoom=7, min_zoom=3,
                                          min_lat=19, max_lat=65,min_lot=-160, max_lot=-70)

# add the facility markers by layers in terms of sourceCategory

sourceCategories = temp_df['sourceCategory'].unique()

for index, sourceCategory in enumerate(sourceCategories):

  facilityGroup_df = temp_df.loc[temp_df.sourceCategory == sourceCategory]
  facilities = [Facility(f) for f in facilityGroup_df.to_dict('records')]
  group = folium.FeatureGroup(sourceCategory).add_to(facility_map)
  iconColor = folium_ICONCOLORS[index]

  for facility in facilities:
    location=[facility.latitude, facility.longitude]
    popup = str(facility)
    icon = folium.Icon(iconColor)
    folium.Marker(location=location,
                  popup=popup, icon=icon).add_to(group)

folium.LayerControl().add_to(facility_map)

#Display the map
facility_map

#  Emissions data

The API query parameters for emissions  data are published [here](https://www.epa.gov/power-sector/cam-api-portal#/swagger/emissions-mgmt).

We will typically be interested in accessing this data by facility at a hourly, daily, monthly, quarterly and annual, which will have the suffix "emissions/apportioned/(hourly|daily|monthly|quarterly|annual)/by-facility"

- stateCode
- facilityId
- unitType
- unitFuelType
- controlTechnologies
- programCodeInfo
- beginDate  
- endDate
- page is page number
- perPage is number of results per page (ignored if page is not set)

Omitting the facilityId will return results for all facilities

Output Schema:
- grossLoad	number Electrical generation in MW produced by combusting a given heat input of fuel.

- steamLoad	number Rate of steam pressure generated by a unit or source produced by combusting a given heat input of fuel. (1000 lb/hr)

- so2Mass	number SO2 Mass Emissions (lbs)

- co2Mass	number CO2 mass emissions (short tons)

- noxMass	number NOx mass emissions (lbs)

- heatInput	number Quantity of heat in mmBtu calculated by multiplying the quantity of fuel by the fuels heat content.

In [17]:
#@title Data loading

dailyEmissionsByFacilityUrl = "https://api.epa.gov/easey/streaming-services/emissions/apportioned/daily/by-facility"

dailyEmissionsByFacility_params = {
    'api_key': API_KEY,
    'beginDate': '2023-01-01',
    'endDate': '2023-12-31',
}  # note that endDate is inclusive

# skip for now
optional_params = {
    # 'stateCode': 'NY',
    # 'facilityId': '3',
    # 'unitType': '',
    # 'unitFuelType': 'Coal|Natural Gas',
    # 'controlTechnologies': '',
    # 'programCodeInfo': 'CAIRNOX', #ARP, CAIRNOX, CAIROS, CAIRSO2, CSNOX, CSNOXOS, CSOSG1, CSOSG2, CSOSG2E, CSOSG3, CSSO2G1, CSSO2G2, NBP, NHNOX, NSPS4T, OTC, RGGI, SIPNOX, TXSO2
    # 'page': 1,
    # 'perPage': 100
}

dailyEmissionsByFacility_params.update(optional_params)

dailyEmissionsByFacility_df = getStreamingResponse(dailyEmissionsByFacilityUrl,dailyEmissionsByFacility_params)
dailyEmissionsByFacility_df['date'] = pd.to_datetime(dailyEmissionsByFacility_df['date'])


print(f"facilities dataset columns: {dailyEmissionsByFacility_df.columns}")



Success: Status code = 200
Field Mappings: [{"label":"State","value":"stateCode"},{"label":"Facility Name","value":"facilityName"},{"label":"Facility ID","value":"facilityId"},{"label":"Date","value":"date"},{"label":"Gross Load (MWh)","value":"grossLoad"},{"label":"Steam Load (1000 lb)","value":"steamLoad"},{"label":"SO2 Mass (short tons)","value":"so2Mass"},{"label":"CO2 Mass (short tons)","value":"co2Mass"},{"label":"NOx Mass (short tons)","value":"noxMass"},{"label":"Heat Input (mmBtu)","value":"heatInput"}]
Returned 475798 rows and 10 columns 
facilities dataset columns: Index(['stateCode', 'facilityName', 'facilityId', 'date', 'grossLoad',
       'steamLoad', 'so2Mass', 'co2Mass', 'noxMass', 'heatInput'],
      dtype='object')


In [41]:
#@title Facility emissions statistics
print(f"No. of unique facilities in dataset: {dailyEmissionsByFacility_df['facilityId'].nunique()}")
print(f"No. of unique dates: {dailyEmissionsByFacility_df['date'].nunique()}")
print(f"Start Date: {dailyEmissionsByFacility_df['date'].min():%Y-%m-%d}")
print(f"End Date: {dailyEmissionsByFacility_df['date'].max():%Y-%m-%d}")

# get aggregate statistics by facility (remove dates for which grossLoad is Nan)

dailyEmissionsByFacility_df = dailyEmissionsByFacility_df[dailyEmissionsByFacility_df['grossLoad'].notna()]

totalEmissionsByFacility_df = dailyEmissionsByFacility_df.groupby(
    by=['facilityId',]).apply(
        lambda df: pd.Series({
          'activeDays':df['date'].nunique(),
          'grossLoad': df["grossLoad"].sum(),
          'so2Mass': df["so2Mass"].sum(),
          'co2Mass': df["co2Mass"].sum(),
          'noxMass': df["noxMass"].sum(),
          })
    ).reset_index()

display(totalEmissionsByFacility_df.sort_values(by='activeDays', ascending=True))

No. of unique facilities in dataset: 1343
No. of unique dates: 365
Start Date: 2023-01-01
End Date: 2023-12-31


,facilityId,activeDays,grossLoad,so2Mass,co2Mass,noxMass
84,673,1.0,83.80,0.000,37.895,0.045
275,2861,2.0,55.80,0.269,0.000,0.646
48,540,2.0,36.62,0.000,0.000,0.233
53,561,2.0,35.58,0.000,0.000,0.423
57,565,2.0,35.36,0.000,0.000,0.420
...,...,...,...,...,...,...
1119,56998,365.0,1290378.93,4.065,800641.823,41.289
603,8023,365.0,4640315.81,1416.898,5067154.754,2178.238
1113,56908,365.0,564568.26,1.915,360862.111,49.633
433,6179,365.0,8269206.89,578.394,9248091.887,5364.129


In [57]:
#@title plot daily emissions

import random
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

# pick a random set of facilities

facilities = [Facility(f) for f in groupedFacilityAttributes_df.to_dict('records')]
facilityChoices = random.choices(facilities, k=15)

@interact
def plotTotalEmissions(facility=facilityChoices):

  facilityEmissions_df = dailyEmissionsByFacility_df[dailyEmissionsByFacility_df['facilityId'] == facility.facilityId]

  print(f"Number of days of data: {facilityEmissions_df[facilityEmissions_df['grossLoad'].notna()].shape[0]}")
  print(totalEmissionsByFacility_df[totalEmissionsByFacility_df['facilityId'] == facility.facilityId].to_string(index=False))

  # first set of figures (Nox,Sox, CO2 emissions versus gross loads)
  fig, axes = plt.subplots(1, 3, sharex=True, figsize=(12,4))
  axes[0].scatter(facilityEmissions_df['grossLoad'],
                  facilityEmissions_df['co2Mass'])
  axes[0].set_xlabel('grossLoad(Mw)')
  axes[0].set_title('co2Mass(short tons)')

  axes[1].scatter(facilityEmissions_df['grossLoad'],
                  facilityEmissions_df['noxMass'])
  axes[1].set_xlabel('grossLoad(Mw)')
  axes[1].set_title('noxMass(lbs)')

  axes[2].scatter(facilityEmissions_df['grossLoad'],
                  facilityEmissions_df['so2Mass'])
  axes[2].set_xlabel('grossLoad(Mw)')
  axes[2].set_title('so2Mass(lbs)')

  fig.suptitle(f"{facility}", fontsize=14)

  plt.show()

  # time series of Nox,Sox, CO2 emissions and gross loads

  print("Time series of emissions and loads")
  facilityTimeSeries_df = dailyEmissionsByFacility_df[dailyEmissionsByFacility_df['facilityId'] == facility.facilityId]
  facilityTimeSeries_df.sort_values(by='date')

  axes = facilityTimeSeries_df.plot(x='date', y=['noxMass','so2Mass','co2Mass', 'grossLoad'],
                            subplots=True, marker='.', linestyle='none',layout=(6,1), figsize=(10,10)).ravel()

  # flatten the array
  axes = axes.flat  # .ravel() and .flatten() also work

  # extract the figure object to use figure level methods
  fig = axes[0].get_figure()

  plt.show()

  #  Nox/grossload vs. CO2/grossload, Nox/grossload vs. CO2/grossload

  print("nox,so2 emissions to c02 emissions on a unit grossload basis")
  fig, axes = plt.subplots(1, 2, sharex=True, figsize=(8,4))
  axes[0].scatter(facilityEmissions_df['co2Mass']/facilityEmissions_df['grossLoad'],
                  facilityEmissions_df['noxMass']/facilityEmissions_df['grossLoad']
                  )
  axes[0].set_xlabel('co2Mass(short tons)/grossLoad(Mw)')
  axes[0].set_ylabel('noxMass(short tons)/grossLoad(Mw)')

  axes[1].scatter(facilityEmissions_df['co2Mass']/facilityEmissions_df['grossLoad'],
                  facilityEmissions_df['so2Mass']/facilityEmissions_df['grossLoad']
                  )
  axes[1].set_xlabel('co2Mass(short tons)/grossLoad(Mw)')
  axes[1].set_ylabel('so2Mass(short tons)/grossLoad(Mw)')

  plt.tight_layout()

  plt.show()

interactive(children=(Dropdown(description='facility', options=(7212 Darbytown Combustion Turbine VA Electric …

# Geometries for  satellite data

- For sentinel-2P images we use a 640X640 sq m region (assuming 10m resolution per pixel this is roughly 64x64 pixels)
- For sentinel-5P NOx data we use 40x40 sq km region (assuming 1123 m per pixel this is roughly 36x36 pixels

### Top 50 facilities by noxMass


In [ ]:


top50byNoxMass_df = totalEmissionsByFacility_df.sort_values(
                                                  by=['noxMass'], ascending=False
                                                  ).iloc[0:50, :].loc[:,'facilityId'].reset_index().drop_duplicates()


top50byNoxMass_df = top50byNoxMass_df.merge(groupedFacilityAttributes_df, how = 'left')
top50byNoxMass_df

,index,facilityId,sourceCategory,nUnits,facilityName,stateCode,countyCode,fipsCode,latitude,longitude,associatedStacks
0,215,2167,Electric Utility,2,New Madrid Power Plant,MO,MO143,143,36.5147,-89.5617,
1,454,6146,Electric Utility,3,Martin Lake,TX,TX401,401,32.2597,-94.5703,
2,439,6076,Electric Utility,2,Colstrip,MT,MT087,087,45.8831,-106.6140,
3,287,2823,Electric Utility,2,Milton R Young,ND,ND065,065,47.0664,-101.2139,
4,216,2168,Electric Utility,3,Thomas Hill Energy Center,MO,MO175,175,39.5531,-92.6392,
5,208,2103,Electric Utility,4,Labadie,MO,MO071,071,38.5583,-90.8361,
6,440,6077,Electric Utility,2,Gerald Gentleman Station,NE,NE111,111,41.0808,-101.1408,
7,639,8042,Electric Utility,2,Belews Creek,NC,NC169,169,36.2811,-80.0603,
8,468,6204,Electric Utility,3,Laramie River,WY,WY031,031,42.1103,-104.8828,
9,494,6705,Industrial Boiler,4,Alcoa Allowance Management Inc,IN,IN173,173,37.9150,-87.3328,


In [ ]:
#@title Facility locations  area of interest.

# size of rectangular geometry centered on the facility over which the satellite data is collected
geometries = {
    's2_geometry': 640,
    's5p_geometry': 40000,
}


import pyproj
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import shapely
import shapely.ops
from shapely.geometry import Point, Polygon

# facilities_df = groupedFacilityAttributes_df[['facilityId', 'facilityName', 'stateCode', 'nUnits', 'sourceCategory',
#                                            'latitude', 'longitude']]

facilities_df = top50byNoxMass_df[['facilityId', 'facilityName', 'stateCode', 'nUnits', 'sourceCategory',
                                           'latitude', 'longitude']].copy()


# add the center point for these new geometries
for key in geometries.keys():
  facilities_df[key] = facilities_df.apply(lambda row:Point(row['longitude'],row['latitude']), axis=1)


# fix the global coordinate system with epsg 4326
GLOBAL_EPSG_CODE = 4326
global_crs = pyproj.CRS(f"EPSG:{GLOBAL_EPSG_CODE}")
# print(global_crs)


# lambda to get the  epsg code for a facility=Id
get_epsg_code = lambda x: int(
          query_utm_crs_info(
          datum_name="WGS 84",
          area_of_interest=AreaOfInterest(
              west_lon_degree=x["longitude"],
              south_lat_degree=x["latitude"],
              east_lon_degree=x["longitude"],
              north_lat_degree=x["latitude"],
          )
        )[0].code)


# iterate over the rows of the data to transform point coordinates to
# rectangular regions of size

rows = [] #initiate empty list for 1st coordinate value
from tqdm import tqdm
for index, row in tqdm(facilities_df.iterrows(), total = facilities_df.shape[0]): #iterate over rows in the dataframe
  # print(row.facilityId)

  #get the local crs
  local_crs = pyproj.CRS(f"EPSG:{get_epsg_code(row)}")

  # Create a PyProj transformer object, convert geometry from global to local_epsg
  transform = pyproj.Transformer.from_crs(global_crs, local_crs, always_xy=True).transform
  for key in geometries.keys():
    row[key] = shapely.ops.transform(transform,row[key])

  # create the required geometries
  for key, val in geometries.items():
    row[key] = row[key].buffer(
            val / 2, cap_style='square'
        )

    # Create a PyProj transformer object, convert geometry from local_epsg to global
  transform_inv = pyproj.Transformer.from_crs(local_crs, global_crs, always_xy=True).transform
  for key in geometries.keys():
    row[key] = shapely.ops.transform(transform_inv,row[key])

  rows.append(row)

facilities_df = pd.DataFrame(pd.concat(rows, axis = 1, ignore_index=True)).transpose()
facilities_df.head(3)


100%|██████████| 50/50 [00:08<00:00,  5.73it/s]


,facilityId,facilityName,stateCode,nUnits,sourceCategory,latitude,longitude,s2_geometry,s5p_geometry
0,2167,New Madrid Power Plant,MO,2,Electric Utility,36.5147,-89.5617,POLYGON ((-89.55822460874718 36.51765863890546...,POLYGON ((-89.34397853573002 36.69943621505673...
1,6146,Martin Lake,TX,3,Electric Utility,32.2597,-94.5703,"POLYGON ((-94.5669534115689 32.2626279579951, ...","POLYGON ((-94.36072420139145 32.4425361119536,..."
2,6076,Colstrip,MT,2,Electric Utility,45.8831,-106.614,POLYGON ((-106.60996085078082 45.8860371952470...,POLYGON ((-106.36073254505438 46.0664065472910...


In [ ]:
#@title Plot area of interest for each facility
import geopandas
import folium
import numpy as np
import matplotlib.colors as mcolors

# create the map
m = folium.Map(location = [40, -102], zoom_start = 4, max_zoom=13, min_zoom=4,
                                          min_lat=19, max_lat=65,min_lot=-160, max_lot=-70)

colors = [c for c in mcolors.BASE_COLORS]

for key in geometries.keys():
  required_cols = ["facilityId",	"latitude",	"longitude"]
  temp_df = pd.concat([facilities_df[required_cols], facilities_df[key]], axis=1)
  temp_gdf = geopandas.GeoDataFrame(temp_df).set_geometry(col=key, crs="EPSG:4326")

  color = np.random.choice(colors)
  m = temp_gdf.explore(m = m, color = color, name = key,
                    popup = ["facilityId",	"latitude",	"longitude"])


folium.LayerControl().add_to(m)

m

# Set up credentials and intialize Google Earth Engine

In [ ]:
from google.colab import auth
from google.api_core import retry
from IPython.display import HTML, Image
from matplotlib import pyplot as plt
from numpy.lib import recfunctions as rfn

import concurrent
import ee
import geemap
import google
import io
import multiprocessing
import numpy as np
import requests
import pandas as pd
import math


# REPLACE WITH YOUR PROJECT!
PROJECT = 'yeshiva-com-4010-spring-2024'
auth.authenticate_user()

credentials, _ = google.auth.default()
ee.Initialize(credentials, project=PROJECT, opt_url='https://earthengine-highvolume.googleapis.com')

# Accesssing Sentinel 2-P data

Code for Sentinel-2 images from  google earth engine

# Accesssing Sentinel-5p data

Code for  Sentinel-5P (NO2) images from  google earth engine

# Accesssing ERA5 climate reanalysis data from Google Earth Engine

Code for  ERA5DailyAggregates data  from  google earth engine